In [1]:
#@title Connecting to google drive (Before Running Main Code Connect to Drive)🥇
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive/')

# # Authenticate Google Drive API
from google.colab import auth
from google.auth import default
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import gspread
auth.authenticate_user() 
creds, _ = default() 
gc = gspread.authorize(creds) 
import gspread_dataframe as gd



import pandas as pd
import shutil
import os  
import datetime
import glob
import re


import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive/


In [2]:
path='/content/drive/MyDrive/MFC1089_Ehthesham_Working_Folder/Order wise sales/Order_Summary_Report_28547_2022-04-01_2022-04-30.csv'

count2=0
_2022_old_format_main_data=pd.DataFrame()

files = glob.glob(os.path.join(path , "*.csv"))

for file in files:
  rd_csv=pd.read_csv(file)
  count2+=1
  rd_csv.insert(0,'Source file',os.path.basename(file),allow_duplicates=False)
  _2022_old_format_main_data=pd.concat([_2022_old_format_main_data,rd_csv])

In [85]:
data=pd.read_csv(path)
data.head(1)

,restaurant_name,invoice_no,date,kot_no,payment_type,order_type,status,sub_order_type,area,virtual_brand_name,...,order_cancel_reason,my_amount,total_tax,discount,delivery_charge,container_charge,service_charge,waived_off,round_off,total
0,Mann Food Court,1,2022-04-01 11:03:13,1,Other [UPI Pay],Pick Up,Success,Pick Up,MFC-Parcel,NaN,...,NaN,289.0,15.0,0.0,0.0,10.0,0,0.0,0.0,314.0


In [86]:
data['date2']=data['date'].astype(str).str.split(" ",expand=True)[0]

In [87]:
condition=data['area'].isin(["MFC-Ground Floor",
"MFC-Parcel",
"MFC-First Floor",
"Magicpin_Dilli Darbar",
"MFC-Home Delivery",
"Magicpin_Valluvan Cafe",
"Magicpin_Mann Food Court",
"Magicpin_Chinese Chaska",
"Home Website",
"Magicpin_Biryani Box"])

data['total']=data['total'].astype(float)

data.loc[condition,'area']='Offline Sales'

data=data[~ (data['status']!='Success')]
data.head()

,restaurant_name,invoice_no,date,kot_no,payment_type,order_type,status,sub_order_type,area,virtual_brand_name,...,my_amount,total_tax,discount,delivery_charge,container_charge,service_charge,waived_off,round_off,total,date2
0,Mann Food Court,1,2022-04-01 11:03:13,1,Other [UPI Pay],Pick Up,Success,Pick Up,Offline Sales,NaN,...,289.0,15.0,0.00,0.0,10.0,0,0.0,0.0,314.0,2022-04-01
1,Mann Food Court,2,2022-04-01 11:12:39,2,Online,Delivery(Parcel),Success,MFC - Roj Ka Khana - Swiggy,Swiggy_MFC - Roj Ka Khana,MFC - Roj Ka Khana,...,216.0,7.4,99.99,0.0,30.0,0,0.0,-0.4,153.0,2022-04-01
2,Mann Food Court,3,2022-04-01 11:15:56,3,Online,Delivery(Parcel),Success,MFC - Dilli Darbar - Swiggy,Swiggy_MFC - Dilli Darbar,MFC - Dilli Darbar,...,195.0,6.8,68.25,0.0,10.0,0,0.0,0.5,144.0,2022-04-01
3,Mann Food Court,4,2022-04-01 11:21:21,4,Online,Delivery(Parcel),Success,MFC - Dilli Darbar - Swiggy,Swiggy_MFC - Dilli Darbar,MFC - Dilli Darbar,...,155.0,6.0,43.39,0.0,10.0,0,0.0,0.4,128.0,2022-04-01
4,Mann Food Court,5,2022-04-01 11:25:39,5,Online,Delivery(Parcel),Success,MFC - Chinese chaska - Zomato,Zomato_MFC - Chinese chaska,MFC - Chinese chaska,...,620.0,24.2,155.00,0.0,20.0,0,0.0,-0.2,509.0,2022-04-01


In [88]:
grouped=data.groupby(['date2','area'])['total'].sum()
grouped=pd.DataFrame(grouped)
grouped=grouped.reset_index()
grouped

,date2,area,total
0,2022-04-01,Amazon_MFC-Alpha Biryani,3250.0
1,2022-04-01,Amazon_MFC-Biryani Box,237.0
2,2022-04-01,Amazon_MFC-Chinese Chaska,747.0
3,2022-04-01,Amazon_MFC-Dilli Darbar,2246.0
4,2022-04-01,Amazon_MFC-Roj Ka Khana,755.0
...,...,...,...
771,2022-04-30,Zomato_MFC - Chinese chaska,2363.0
772,2022-04-30,Zomato_MFC - Roj Ka Khana,3339.0
773,2022-04-30,Zomato_MFC - Valluvan Cafe,1491.0
774,2022-04-30,Zomato_MFC-MANN FOOD COURT,2535.0


In [89]:
#@title go

offline_sales=grouped[grouped['area'].isin(['Offline Sales'])]



#offline_sales['Brand_code']=offline_sales['Res_Name'].map(brand_code)

#offline_sales['Invoice Number']='ZM'+offline_sales['Brand_code']+l2+l1


offline_sales.rename(columns={"date2":'Invoice Date'},inplace=True)

offline_sales['Item Name']='Food Sales'


In [93]:
#@title go2

offline_sales["Customer Name"]	=	'Offline Sales'
offline_sales["Invoice Status"]	=	"Overdue"
offline_sales["GST Treatment"]	=	"Consumer"
offline_sales["Place of Supply"]	=	"KA"
offline_sales["Currency Code"]	=	"INR"
offline_sales["Exchange Rate"]	=	"1"
offline_sales["Account"]	=	"Sales"
offline_sales["Item Type"]	=	"Goods"
offline_sales["Quantity"]	=	"1"
offline_sales["Usage unit"]	=	"Count"
offline_sales["Is Inclusive Tax"]	=	"Tax"
offline_sales["Item Tax"]	=	"GST5"
offline_sales["Item Tax Type"]	=	"Tax Group"
offline_sales["Item Tax %"]	=	"0%"
offline_sales["Supply Type"]	=	"Taxable"
offline_sales["Discount Type"]	=	"entity_level"
offline_sales["Adjustment"]	=	"Adjustment"
offline_sales.rename(columns={'total':"Item Price"},inplace=True)


offline_sales['Item Price']=offline_sales['Item Price'].astype(float)
offline_sales['Item Price'] = offline_sales['Item Price']/1.05



f = {'01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun', '07': 'July', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'}

offline_sales['mont']=offline_sales['Invoice Date'].str.split("-",expand=True)[1].map(f)


offline_sales['Invoice Number'] = 'MHL-'+offline_sales['Invoice Date'].str.split("-",expand=True)[2]+offline_sales['mont']+offline_sales['Invoice Date'].str.split("-",expand=True)[0]

offline_sales.drop(columns=['mont','area'],inplace=True)

In [94]:
offline_sales.head(1)

,Invoice Date,Item Price,Item Name,Customer Name,Invoice Status,GST Treatment,Place of Supply,Currency Code,Exchange Rate,Account,...,Quantity,Usage unit,Is Inclusive Tax,Item Tax,Item Tax Type,Item Tax %,Supply Type,Discount Type,Adjustment,Invoice Number
10,2022-04-01,28842.673577,Food Sales,Offline Sales,Overdue,Consumer,KA,INR,1,Sales,...,1,Count,Tax,GST5,Tax Group,0%,Taxable,entity_level,Adjustment,MHL-01Apr2022


In [95]:
offline_sales.to_excel('Sales5.xlsx',index=False)